In [ ]:
pip install roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
from roboflow import Roboflow
import math

# Roboflow setup
rf = Roboflow(api_key="HSvdWoVcQ0H4zdWTiCTN")
project = rf.workspace().project("weld-dimension-prediction")
model = project.version(1).model  # Replace "1" with your model version

# Infer on a test image
test_image_path = "/content/WIN_20240902_19_35_26_Pro.jpg"  # Path to your test image
segmentation_result = model.predict(test_image_path).json()

# Save the annotated image for visualization
model.predict(test_image_path).save("prediction.jpg")


loading Roboflow workspace...
loading Roboflow project...


In [ ]:
# Parse the segmentation results
scale_polygon = None
weld_polygon = None

# Extract polygons for scale and weld
for obj in segmentation_result["predictions"]:
    if obj["class"] == "scale":
        scale_polygon = obj["points"]
    elif obj["class"] == "weld":
        weld_polygon = obj["points"]
print(segmentation_result)



{'predictions': [{'x': 1060.0, 'y': 664.0, 'width': 736.0, 'height': 76.0, 'confidence': 0.8819184303283691, 'class': 'ruler', 'points': [{'x': 696.0, 'y': 627.75}, {'x': 693.0, 'y': 629.4375}, {'x': 693.0, 'y': 686.8125}, {'x': 708.0, 'y': 686.8125}, {'x': 711.0, 'y': 688.5}, {'x': 723.0, 'y': 688.5}, {'x': 726.0, 'y': 690.1875}, {'x': 729.0, 'y': 690.1875}, {'x': 732.0, 'y': 691.875}, {'x': 753.0, 'y': 691.875}, {'x': 756.0, 'y': 693.5625}, {'x': 1143.0, 'y': 693.5625}, {'x': 1146.0, 'y': 695.25}, {'x': 1155.0, 'y': 695.25}, {'x': 1158.0, 'y': 696.9375}, {'x': 1161.0, 'y': 696.9375}, {'x': 1164.0, 'y': 698.625}, {'x': 1176.0, 'y': 698.625}, {'x': 1179.0, 'y': 700.3125}, {'x': 1422.0, 'y': 700.3125}, {'x': 1425.0, 'y': 698.625}, {'x': 1425.0, 'y': 644.625}, {'x': 1419.0, 'y': 641.25}, {'x': 1344.0, 'y': 641.25}, {'x': 1341.0, 'y': 639.5625}, {'x': 1314.0, 'y': 639.5625}, {'x': 1308.0, 'y': 636.1875}, {'x': 1311.0, 'y': 634.5}, {'x': 1320.0, 'y': 634.5}, {'x': 1323.0, 'y': 632.8125}, {

In [ ]:
# Extract the ruler (scale) and weld polygons
ruler_polygon = next((obj["points"] for obj in segmentation_result["predictions"] if obj["class"] == "ruler"), None)
weld_polygon = next((obj["points"] for obj in segmentation_result["predictions"] if obj["class"] == "welds"), None)

if ruler_polygon is None or weld_polygon is None:
    raise ValueError("Ruler or weld not detected in the segmentation output.")

# Helper function to calculate Euclidean distance
def calculate_length(polygon):
    x_coords = [point["x"] for point in polygon]
    y_coords = [point["y"] for point in polygon]
    return max(((x_coords[i] - x_coords[j]) ** 2 + (y_coords[i] - y_coords[j]) ** 2) ** 0.5 for i in range(len(polygon)) for j in range(i + 1, len(polygon)))

# Calculate pixel lengths
ruler_length_pixels = calculate_length(ruler_polygon)
weld_length_pixels = calculate_length(weld_polygon)

# Convert pixel length to cm using the real-life ruler length (31.5 cm)
pixel_to_cm_ratio = 31.5 / ruler_length_pixels
weld_length_cm = weld_length_pixels * pixel_to_cm_ratio

print(f"Weld length in cm: {weld_length_cm}")


Weld length in cm: 18.00286703570301
